<a href="https://colab.research.google.com/github/AnxietySquid/test_tg_bot/blob/main/HW12_2_eremin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q aiogram
!pip install -q nest_asyncio
!pip install -q aiosqlite

In [2]:
import nest_asyncio
nest_asyncio.apply()
from aiogram.filters.command import Command
from aiogram.utils.keyboard import InlineKeyboardBuilder, ReplyKeyboardBuilder
import asyncio
import logging
from aiogram import Bot, Dispatcher, types, F


In [3]:
import aiosqlite

DB_NAME = 'quiz_bot.db'
async def create_table():
    # Создаем соединение с базой данных (если она не существует, то она будет создана)
    async with aiosqlite.connect(DB_NAME) as db:
        # Выполняем SQL-запрос к базе данных
        await db.execute('''CREATE TABLE IF NOT EXISTS quiz_state (user_id INTEGER PRIMARY KEY, question_index INTEGER, user_score INTEGER)''')
        # Сохраняем изменения
        await db.commit()

# Запускаем создание таблицы базы данных
await create_table()

In [4]:
async def update_quiz_index(user_id, index):
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect(DB_NAME) as db:
        # Вставляем новую запись или заменяем ее, если с данным user_id уже существует
        await db.execute('INSERT OR REPLACE INTO quiz_state (user_id, question_index) VALUES (?, ?)', (user_id, index))
        # Сохраняем изменения
        await db.commit()

In [5]:
async def get_quiz_index(user_id):
     # Подключаемся к базе данных
     async with aiosqlite.connect(DB_NAME) as db:
        # Получаем запись для заданного пользователя
        async with db.execute('SELECT question_index FROM quiz_state WHERE user_id = (?)', (user_id, )) as cursor:
            # Возвращаем результат
            results = await cursor.fetchone()
            if results is not None:
                return results[0]
            else:
                return 0

In [24]:
async def update_quiz_data(user_id, index, user_score):
    # Создаем соединение с базой данных (если она не существует, она будет создана)
    async with aiosqlite.connect(DB_NAME) as db:
        # Вставляем новую запись или заменяем ее, если с данным user_id уже существует
        await db.execute('INSERT OR REPLACE INTO quiz_state (user_id, question_index, user_score) VALUES (?, ?, ?)', (user_id, index, user_score))
        # Сохраняем изменения
        await db.commit()

In [26]:
async def get_quiz_score(user_id):
    async with aiosqlite.connect(DB_NAME) as db:
        async with db.execute('SELECT user_score FROM quiz_state WHERE user_id = ?', (user_id,)) as cursor:
            result = await cursor.fetchone()
            if result[0] is not None:
              return int(result[0])
            else:
              return 0


In [8]:
#@title questions
quiz_data = [
    {
        'question': 'Что такое Python?',
        'options': ['Язык программирования', 'Тип данных', 'Музыкальный инструмент', 'Змея на английском'],
        'correct_option': 0
    },
    {
        'question': 'Какой тип данных используется для хранения целых чисел?',
        'options': ['int', 'float', 'str', 'natural'],
        'correct_option': 0
    },
    {
        'question': 'В чем разница между типами данных "array" и "list" в Python?',
        'options': ['Массивы используются для арифметических операций, в то время как списки нет', 'Нет разницы; они одинаковы в Python', 'Списки изменяемы, в то время как массивы неизменяемы', 'Массивы могут содержать элементы только одного типа данных, в то время как списки могут содержать элементы разных типов данных'],
        'correct_option': 3
    },
    {
        'question': 'Для чего обычно используется "matplotlib" в Python?',
        'options': ['Для визуализации данных', 'Для моделей машинного обучения', 'Для веб-скрейпинга', 'Для управления базами данных'],
        'correct_option': 0
    },
    {
        'question': 'Какой тип данных в Python является упорядоченным и неизменяемым?',
        'options': ['tuple', 'list', 'set', 'dict'],
        'correct_option': 2
    },
    {
        'question': 'Какое из следующих не является допустимым именем переменной в Python?',
        'options': ['2variable', 'my_variable', '_variable', 'variable2'],
        'correct_option': 0
    },
    {
        'question': 'Для чего используется срез (slicing) в Python?',
        'options': ['Для извлечения частей последовательностей, таких как списки или строки', 'Для удаления элементов из списка', 'Для добавления элементов в список', 'Для сортировки элементов списка'],
        'correct_option': 0
    },
    {
        'question': 'Как сконкатенировать два списка в Python?',
        'options': ['Используя метод append()', 'Используя оператор "*"', 'Используя оператор "+"', 'Используя метод concat()'],
        'correct_option': 2
    },
    {
        'question': 'Для чего в Python используется конструкция "with" при работе с файлами?',
        'options': ['Для удаления файла', 'Для записи данных в файл', 'Для чтения данных из файла', 'Для обеспечения правильного управления ресурсами и автоматического закрытия файла'],
        'correct_option': 3
    },
    {
        'question': 'Каков результат "set([1, 2, 3]) & set([2, 3, 4])" в Python?',
        'options': ['set([1, 2, 3, 4])', 'set([2, 3])', 'set([1, 4])', 'Ошибка'],
        'correct_option': 1
    },
    # Добавьте другие вопросы
]

In [27]:
# Включаем логирование, чтобы не пропустить важные сообщения
logging.basicConfig(level=logging.INFO)

# Замените "YOUR_BOT_TOKEN" на токен, который вы получили от BotFather
API_TOKEN = '7967200274:AAGHujEyJ5gIqlV9SDnLnxo70HWzvPbo0_A'


# Объект бота
bot = Bot(token=API_TOKEN)
# Диспетчер
dp = Dispatcher()


# Хэндлер на команду /start
@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    # Создаем сборщика клавиатур типа Reply
    builder = ReplyKeyboardBuilder()
    # Добавляем в сборщик одну кнопку
    builder.add(types.KeyboardButton(text="Начать игру"))
    # Прикрепляем кнопки к сообщению
    await message.answer("Добро пожаловать в квиз!", reply_markup=builder.as_markup(resize_keyboard=True))

async def new_quiz(message):
    # получаем id пользователя, отправившего сообщение
    user_id = message.from_user.id
    # сбрасываем значение текущего индекса вопроса квиза в 0
    current_question_index = 0
    user_score = 0
    await update_quiz_data(user_id, current_question_index, user_score)

    # запрашиваем новый вопрос для квиза
    await get_question(message, user_id)

def generate_options_keyboard(answer_options, right_answer):
  # Создаем сборщика клавиатур типа Inline
    builder = InlineKeyboardBuilder()

    # В цикле создаем 4 Inline кнопки, а точнее Callback-кнопки
    for option in answer_options:
        builder.add(types.InlineKeyboardButton(
            # Текст на кнопках соответствует вариантам ответов
            text=option,
            # Присваиваем данные для колбэк запроса.
            # Если ответ верный сформируется колбэк-запрос с данными 'right_answer'
            # Если ответ неверный сформируется колбэк-запрос с данными 'wrong_answer'
            callback_data="right_answer" if option == right_answer else "wrong_answer")
        )

    # Выводим по одной кнопке в столбик
    builder.adjust(1)
    return builder.as_markup()

@dp.callback_query(F.data == "right_answer")
async def right_answer(callback: types.CallbackQuery):
    # редактируем текущее сообщение с целью убрать кнопки (reply_markup=None)
    await callback.bot.edit_message_reply_markup(
        chat_id=callback.from_user.id,
        message_id=callback.message.message_id,
        reply_markup=None
    )

    # Получение текущего вопроса для данного пользователя
    current_question_index = await get_quiz_index(callback.from_user.id)

    # Отправляем в чат сообщение, что ответ верный
    await callback.message.answer(f"Ваш ответ: {quiz_data[current_question_index]['options'][quiz_data[current_question_index]['correct_option']]}\n Верно! 🎉")

    # Обновление номера текущего вопроса в базе данных
    current_question_index += 1

    # Добавляем очко
    user_score = await get_quiz_score(callback.from_user.id)
    user_score += 1
    await update_quiz_data(callback.from_user.id, current_question_index, user_score)

    # Проверяем достигнут ли конец квиза
    if current_question_index < len(quiz_data):
        # Следующий вопрос
        await get_question(callback.message, callback.from_user.id)
    else:
        # Уведомление об окончании квиза
        await callback.message.answer("Это был последний вопрос. Квиз завершен!")
        await callback.message.answer(f"Ваш результат: {user_score}/{len(quiz_data)}")

@dp.callback_query(F.data == "wrong_answer")
async def wrong_answer(callback: types.CallbackQuery):
    # редактируем текущее сообщение с целью убрать кнопки (reply_markup=None)
    await callback.bot.edit_message_reply_markup(
        chat_id=callback.from_user.id,
        message_id=callback.message.message_id,
        reply_markup=None
    )

    # Получение текущего вопроса для данного пользователя
    current_question_index = await get_quiz_index(callback.from_user.id)

    correct_option = quiz_data[current_question_index]['correct_option']

    # Отправляем в чат сообщение об ошибке с указанием верного ответа
    await callback.message.answer(f"Неправильно. Правильный ответ: {quiz_data[current_question_index]['options'][correct_option]}")

    # Обновление номера текущего вопроса в базе данных
    current_question_index += 1

    # Добавляем очко
    user_score = await get_quiz_score(callback.from_user.id)
    await update_quiz_data(callback.from_user.id, current_question_index, user_score)

    # Проверяем достигнут ли конец квиза
    if current_question_index < len(quiz_data):
        # Следующий вопрос
        await get_question(callback.message, callback.from_user.id)
    else:
        # Уведомление об окончании квиза
        await callback.message.answer("Это был последний вопрос. Квиз завершен!")
        await callback.message.answer(f"Ваш результат: {user_score}/{len(quiz_data)}")

async def get_question(message, user_id):

    # Запрашиваем из базы текущий индекс для вопроса
    current_question_index = await get_quiz_index(user_id)
    # Получаем индекс правильного ответа для текущего вопроса
    correct_index = quiz_data[current_question_index]['correct_option']
    # Получаем список вариантов ответа для текущего вопроса
    opts = quiz_data[current_question_index]['options']

    # Функция генерации кнопок для текущего вопроса квиза
    # В качестве аргументов передаем варианты ответов и значение правильного ответа (не индекс!)
    kb = generate_options_keyboard(opts, opts[correct_index])
    # Отправляем в чат сообщение с вопросом, прикрепляем сгенерированные кнопки
    await message.answer(f"{quiz_data[current_question_index]['question']}", reply_markup=kb)

# Хэндлер на команду /quiz
@dp.message(F.text=="Начать игру")
@dp.message(Command("quiz"))
async def cmd_quiz(message: types.Message):
    # Отправляем новое сообщение без кнопок
    await message.answer(f"Давайте начнем квиз!")
    # Запускаем новый квиз
    await new_quiz(message)


# Запуск процесса поллинга новых апдейтов
async def main():
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())